# Abstract 

La posibilidad que brindan los sistemas automatizados de decision y clasificacion son sobresalientes al producir mejoras de producto y reducir costos o aumentar ganacias, de manera sistematica y eficaz. Muchas empresas relacionadas al juego y apuestas utilizan esto para predecir numeros ante la gran cantidad de datos que se nos brinda dados en el dataset. Asi mismo , el cliente propone una solucion practica ante las circunstacias economicas que atraviesa.

En base a esta informacion , se puede hacer una investigacion y resolver las practicas beneficiosas de los jugadores y del equipo en cada uno de los juegos, es decir , es posible hacer preguntas y responderlas para obtener nuevos conocimientos sobre los equipos, los jugadores y la competencia en sí. Por ejemplo, se puede analizar la tendencia de los equipos en casa y fuera de casa, las estadísticas de los jugadores en diferentes momentos del partido, o el porcentaje de victorias y derrotas de los equipos.

Este proyecto se basara en utilizar los datos disponibles para encontrar las diferencias y probrabilidades en cada partido dado durante los años jugados por la NBA. El objetivo se centrara en poder adquirir una variable que determine un numero en el cual se basara la cuota cobrada por el sistema de apuestas. Ademas , este analisis proovera al cliente con el gran beneficio del conocimiento y la ventaja por sobre la competencia , o hacia los mismos usuarios de la plataforma.

Asi, lo visto durante un analisis exploratorio de los datos y las caracteristicas del conjunto , se plantearan las primeras hipotesis para luego establecer un modelo predictivo de los datos de interes. Luego de un resultado final , se podra verificar el uso de las herramientas y las capacidades un sistema de ML.

## Analisis exploratorio complementario (Adquisicion de datos por APIs)

Para verificar con mayor veracidad los datos , podria utilizarse alguna API que nos permita obtener datos actualizados acerca de los juegos que no se encuentran en el dataset. Estos datos nuevos podrian utilizarse en la parte de validacion del modelo o para sacar nuevas hipotesis en base a las tendencias de los nuevos juegos. 

Hay que comentar que estos tipos de datos suelen ser de pago y no siempre se encuentran de manera libre o por "suscripcion gratuita". Lo ideal seria utilizar APIs que sean oficiales , limpias de datos erroneos o ordenados de manera correcta y sistematica.

En esta oportunidad utilizaremos la API publica brindada por *"Ball don't lie"* , que a pesar de ser una API NO OFICIAL, nos permitira obtener algunos datos relevantes de las temporadas que no tenemos en el dataset de nuestro proyecto. Su website es "https://www.balldontlie.io/home.html#introduction" en la que se encuentra la manera de realizar "querys" a su url.

Recuerde que los datos del dataset provisto durante las etapas anteriores , alcanzan las temporadas 2003-2021 (Ultima fecha registrada = 2022-03-12)

Como se nombro anteriormente , esta informacion se utilizara para complementar los datos de las temporadas anteriores , por lo tanto , se procedera a realizar un "GET" para la informacion de la temporadara ("SEASON") 2022.

In [62]:
import requests

url = "https://www.balldontlie.io/api/v1/games?seasons[]=2022&per_page=100"

response = requests.get(url)
data = response.json()
data['meta']

{'total_pages': 14,
 'current_page': 1,
 'next_page': 2,
 'per_page': 100,
 'total_count': 1307}

Al ser demasiada informacion para procesar , utilizan un sistema de paginas que proveen la informacion por "bloques". En el apartado ['meta'] , nos provee de la informacion que se obtiene del uso de la API , que nos informa que existen **14 paginas** y **1307 claves** para esta temporada.

Se utilizara 100 resultados por pagina ya que es el limite que dispone la documentacion de la API y de esta manera , sera necesario acceder a las 14 paginas para completar el DF con todos los datos. Una manera de hacerlo es iterar sobre la utilizacion de la API y asi obtener la informacion completa.

In [65]:
import pandas as pd

#lista para almacenar los diccionarios de estadísticas
stats_list = []

for i in range (1,15):
    #global stats_list
    url = 'https://www.balldontlie.io/api/v1/games?seasons[]=2022&per_page=100&page=' + str(i)
    response = requests.get(url)
    data = response.json()
    # Iterar sobre cada partido en la pagina y agregar las estadísticas como columnas en el DataFrame
    for game in data['data']:
        stats = {'game_id': game['id'],
                'game_date' : game['date'],
                'game_home_team' : game['home_team']['name'],
                'game_home_team_score' : game['home_team_score'],
                'game_season' : game['season'],
                'game_visitor_team' : game['visitor_team']['name'],
                'game_visitor_team_score' : game['visitor_team_score']}
        stats_list.append(stats)
    

# Ordenar la lista de diccionarios por "game_id"
sorted_stats_list = stats_list.sort(key=lambda x: x['game_id'])

df = pd.DataFrame(stats_list)
df


,game_id,game_date,game_home_team,game_home_team_score,game_season,game_visitor_team,game_visitor_team_score
0,857355,2022-10-18T00:00:00.000Z,Celtics,126,2022,76ers,117
1,857356,2022-10-18T00:00:00.000Z,Warriors,123,2022,Lakers,109
2,857357,2022-10-19T00:00:00.000Z,Pacers,107,2022,Wizards,114
3,857358,2022-10-19T00:00:00.000Z,Pistons,113,2022,Magic,109
4,857359,2022-10-19T00:00:00.000Z,Nets,108,2022,Pelicans,130
...,...,...,...,...,...,...,...
1302,981619,2023-05-08T00:00:00.000Z,Lakers,104,2022,Warriors,101
1303,981623,2023-05-10T00:00:00.000Z,Warriors,121,2022,Lakers,106
1304,981627,2023-05-12T00:00:00.000Z,Lakers,0,2022,Warriors,0
1305,1006710,2023-05-14T00:00:00.000Z,Warriors,0,2022,Lakers,0


Como se puede ver , el DF generado contiene los registros de toda la temporada 2022. Estos datos fueron ordenados por **'game_id'** , ya que la API los entregaba de manera desornada.

Pasaremos a hacer una limpieza de los mismos y un analisis exploratorio para el futuro uso de los mismos. 

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1307 entries, 0 to 1306
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   game_id                  1307 non-null   int64 
 1   game_date                1307 non-null   object
 2   game_home_team           1307 non-null   object
 3   game_home_team_score     1307 non-null   int64 
 4   game_season              1307 non-null   int64 
 5   game_visitor_team        1307 non-null   object
 6   game_visitor_team_score  1307 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 71.6+ KB


No se encontraron valores nulos entre los datos

In [67]:
df.describe()

,game_id,game_home_team_score,game_season,game_visitor_team_score
count,1.307000e+03,1307.000000,1307.0,1307.000000
mean,8.618814e+05,115.085692,2022.0,112.449885
std,1.832810e+04,14.926374,0.0,14.851987
min,8.573550e+05,0.000000,2022.0,0.000000
25%,8.576815e+05,108.000000,2022.0,105.000000
50%,8.580080e+05,116.000000,2022.0,113.000000
75%,8.583345e+05,124.000000,2022.0,121.000000
max,1.006711e+06,175.000000,2022.0,176.000000


A pesar de que hay posibilidad de que existan partidos en los que haya 0 anotaciones (game_home_team_score , game_visitor_team_score) , es sumamente improbable. Por lo tanto se tomara las filas estas como **datos erroneos o faltantes**. Se contaran cuantos registros cumplen con estas condiciones

In [72]:
# Filtrar las filas que cumplen con la condición de que "game_home_team_score" sea igual a 0 o "game_visitor_team_score" sea igual a 0
filtered_df = df.loc[(df['game_home_team_score'] == 0) | (df['game_visitor_team_score'] == 0)]
filtered_df


,game_id,game_date,game_home_team,game_home_team_score,game_season,game_visitor_team,game_visitor_team_score
1284,967809,2023-05-11T00:00:00.000Z,Suns,0,2022,Nuggets,0
1285,967810,2023-05-14T00:00:00.000Z,Nuggets,0,2022,Suns,0
1291,970441,2023-05-12T00:00:00.000Z,Heat,0,2022,Knicks,0
1297,972866,2023-05-11T00:00:00.000Z,76ers,0,2022,Celtics,0
1298,972869,2023-05-14T00:00:00.000Z,Celtics,0,2022,76ers,0
1304,981627,2023-05-12T00:00:00.000Z,Lakers,0,2022,Warriors,0
1305,1006710,2023-05-14T00:00:00.000Z,Warriors,0,2022,Lakers,0
1306,1006711,2023-05-15T00:00:00.000Z,Knicks,0,2022,Heat,0


Como se puede ver , los casos que son nulos son pocos y no afectan realmente a los datos , por lo tanto , se procedera con la eliminacion de estos registros. El siguiente codigo es una manera distinta de eliminacion de los registros aunque igual de eficaz

In [75]:
df = df.loc[(df['game_home_team_score'] != 0) | (df['game_visitor_team_score'] != 0)]
df.describe()

,game_id,game_home_team_score,game_season,game_visitor_team_score
count,1299.000000,1299.000000,1299.0,1299.000000
mean,861148.689761,115.794457,2022.0,113.142417
std,15771.402805,11.917866,0.0,11.980231
min,857355.000000,80.000000,2022.0,79.000000
25%,857679.500000,108.000000,2022.0,105.000000
50%,858004.000000,116.000000,2022.0,113.000000
75%,858328.500000,124.000000,2022.0,121.000000
max,981623.000000,175.000000,2022.0,176.000000


Finalmente, el Dataframe ya posee datos utlizables y accesibles de manera confiable. Se continuara con el analisis exploratorio del mismo y de comparacion con los datos del dataset del proyecto 